In [1]:
from geopy.distance import geodesic
from itertools import permutations
import pandas as pd

In [2]:
def calculate_distance(coord1: float, coord2: float) -> float:
    """
    Função para calcular a distância entre dois pontos
    """
    return geodesic(coord1, coord2).kilometers

In [3]:
def find_nearby_places(current_location: tuple, df, radius_km: float):
    nearby_places = []
    for _, row in df.iterrows():
        place_location = (row['Latitude'], row['Longitude'])
        distance = geodesic(current_location, place_location).kilometers

        if distance <= radius_km:
            nearby_places.append({
                'Descrição': row['Descrição'],
                'Distância em km': round(distance, 2)
            })
    return nearby_places

In [4]:
def calculate_shortest_route(points: list, back_to_starting_point: bool = False):
    """
    Calcula a rota mais curta entre uma lista de pontos usando a distância geodésica.
    
    :param points: Lista de tuplas (latitude, longitude)
    :back_to_starting_point: booleano para adicionar a distância de volta ao ponto inicial
    :return: Tupla contendo a ordem dos pontos na rota mais curta e a distância total
    """
    if len(points) < 2:
        raise ValueError('A lista de pontos deve conter pelo menos dois pontos.')

    # somente pontos exclusivos, sem duplicidade
    points = list(set(points))

    shortest_distance = float('inf')
    shortest_route = None
    
    # Gera todas as permutações (variações) possíveis dos pontos
    for route in permutations(points):
        total_distance = 0
        
        # Calcula a distância total da rota atual
        for i in range(len(route) - 1):
            total_distance += geodesic(route[i], route[i+1]).kilometers
        
        # Adiciona a distância de volta ao ponto inicial
        if back_to_starting_point:
            total_distance += geodesic(route[-1], route[0]).kilometers
        
        # Atualiza a rota mais curta se encontrarmos uma melhor
        if total_distance < shortest_distance:
            shortest_distance = total_distance
            shortest_route = route
    
    return shortest_route, shortest_distance

In [5]:
df = pd.DataFrame({
    'Descrição':['Casa das Rosas', 'Japan House São Paulo', 'SESC Avenida Paulista', 'Itaú Cultural', 'Gazeta'],
    'Endereço':['Av. Paulista, 37 - Bela Vista, São Paulo - SP, 01311-000', 'Av. Paulista, 52 - Bela Vista, São Paulo - SP, 01310-900', 'Av. Paulista, 119 - Bela Vista, São Paulo - SP, 01311-903', 'Av. Paulista, 149 - Bela Vista, São Paulo - SP, 01311-000', 'Av. Paulista, 900 - Bela Vista, São Paulo - SP, 01310-100'],
    'Latitude':['-23.571287', '-23.570591', '-23.570569', '-23.56951', '-23.565415'],
    'Longitude':['-46.645466', '-46.644805', '-46.645460', '-46.64612', '-46.651396']})
df

,Descrição,Endereço,Latitude,Longitude
0,Casa das Rosas,"Av. Paulista, 37 - Bela Vista, São Paulo - SP,...",-23.571287,-46.645466
1,Japan House São Paulo,"Av. Paulista, 52 - Bela Vista, São Paulo - SP,...",-23.570591,-46.644805
2,SESC Avenida Paulista,"Av. Paulista, 119 - Bela Vista, São Paulo - SP...",-23.570569,-46.645460
3,Itaú Cultural,"Av. Paulista, 149 - Bela Vista, São Paulo - SP...",-23.56951,-46.64612
4,Gazeta,"Av. Paulista, 900 - Bela Vista, São Paulo - SP...",-23.565415,-46.651396


In [7]:
current_location = (-23.568869, -46.639925) # Metrô Vergueiro
radius_km = 0.7
nearby_places = find_nearby_places(current_location, df, radius_km)

if nearby_places:
    print(f'Lugares com menos de {radius_km} km de distância da localização atual:')
    for place in nearby_places:
        print(f'- {place['Descrição']}, {place['Distância em km']} km de distância')
else:
    print(f'Não há lugares próximos em volta de {radius_km} km da localização.')

Lugares com menos de 0.7 km de distância da localização atual:
- Casa das Rosas, 0.63 km de distância
- Japan House São Paulo, 0.53 km de distância
- SESC Avenida Paulista, 0.6 km de distância
- Itaú Cultural, 0.64 km de distância


In [9]:
pontos = list(zip(df['Latitude'], df['Longitude']))
distancia_total = 0
for i in range(len(pontos) - 1):
    distancia_total += geodesic(pontos[i], pontos[i+1]).km
distancia_total

1.0087516626409072

In [10]:
shortest_route, shortest_distance = calculate_shortest_route(points=pontos)
print(f"Rota mais curta: {shortest_route}")
print(f"Distância total: {shortest_distance:.2f} km")

Rota mais curta: (('-23.571287', '-46.645466'), ('-23.570591', '-46.644805'), ('-23.570569', '-46.645460'), ('-23.56951', '-46.64612'), ('-23.565415', '-46.651396'))
Distância total: 1.01 km


In [11]:
sorted(pontos, key=lambda x: calculate_distance((-23.556175, -46.662075), x))

[('-23.565415', '-46.651396'),
 ('-23.56951', '-46.64612'),
 ('-23.570569', '-46.645460'),
 ('-23.570591', '-46.644805'),
 ('-23.571287', '-46.645466')]